In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

meshwidths = np.array([1.0,0.5,0.25]) 
minEV = np.zeros(len(meshwidths))

k = 0

netgen_mesh =unit_cube.GenerateMesh(maxh=1)
mesh=Mesh(netgen_mesh)

for h in meshwidths:
    print("h=",h, "\n")
    #Draw(mesh)

    #set function space
    H_h = Compress(L2(mesh, order = 1, complex=True)) # main function space
    H_H = Compress(L2(mesh, order = 2, complex=True)) # high order Function space for Riesz representative 
    
    
    #choose w_h and set its values on the nodes
    # w = exp(2j * z)

    # w_h = GridFunction(H_h) #set the nodes
    # w_h.Set(w)

    # f_H = GridFunction(H_H, name="f_H") #high order 


    #low order trial and test function
    u_h = H_h.TrialFunction() 
    v_h = H_h.TestFunction() 
    #High order trial and test function
    u_H = H_H.TrialFunction() 
    v_H= H_H.TestFunction() 

    m = BilinearForm(H_H) #define m with the L2 inner product on the high order space
    m += u_H*v_H * dx 
    m.Assemble()

    a = BilinearForm(H_H) # define the L2 inner product on the high order space
    a += u_H*v_H * dx 
    a.Assemble()


    #embedding matrix t
    a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)
    a_mixed += u_h*v_H*dx
    a_mixed.Assemble()

    a_dest = BilinearForm(H_H)
    a_dest += u_H*v_H*dx
    a_dest.Assemble()

    t = a_dest.mat.Inverse() @ a_mixed.mat

    #w_H is the high order embedding of w_h
    # w_H = GridFunction(H_H, name="w_H")
    # w_H.vec.data = t * w_h.vec

    #compute riesz representative
    a_inv = a.mat.Inverse(freedofs=H_H.FreeDofs())

    #solve for the riesz representative (f_H) 
    # temp = f_H.vec.CreateVector()
    # temp.data = m.mat * w_H.vec
    # f_H.vec.data = a_inv * temp

    #create Matrices for GEV problem

    #c is the low order galerkin matrix
    c = BilinearForm(H_h)
    c += u_h*v_h * dx 
    c.Assemble()

    #check norms
    # temp3 = c.mat * w_h.vec
    # w_h_norm = w_h.vec.InnerProduct(temp3)
    # print("L2 norm of w_h:", sqrt(w_h_norm)) #norm of w_h in H

    # temp4 = a.mat * f_H.vec
    # f_H_norm = f_H.vec.InnerProduct(temp4)
    # print("L2 norm of f_H, the riesz representative of w_h:", sqrt(f_H_norm)) #norm of the riesz representative

    C = c.mat.ToDense().NumPy()
    M = m.mat.ToDense().NumPy()
    A_inv = a_inv.ToDense().NumPy()
    T = t.ToDense().NumPy()

    #M == M.T 
    B = T.T @ M.T @ A_inv @ M @ T

    #B,C are symmetric
    EV = sp.linalg.eigvalsh(B,C)
    lam_max = max(EV)
    print(lam_max)
    lam_max_abs = np.abs(lam_max)
    print(lam_max_abs)

    lam=min(EV)
    print(lam)
    lam_abs = np.abs(lam)
    print(lam_abs)

    minEV[k] = lam_abs
    
    k = k + 1
    mesh.Refine()
    print("\n")

fig, ax = plt.subplots()
plt.grid(visible=True)
plt.loglog(meshwidths,minEV, 'o-', label="minimal eigenvalues")
plt.loglog(meshwidths,np.power(meshwidths,2),'o-', label="O(h^2)")
plt.loglog(meshwidths,np.power(meshwidths,3),'o-', label="O(h^3)")
plt.loglog(meshwidths,np.power(meshwidths,4),'o-', label="O(h^4)")
plt.xlabel('meshwidth h')
plt.ylabel('minimal Eigenvalue')
plt.title('d=3, l=3')
plt.legend()
plt.show()
